# nbscholar 大牛学者

> nbdev Extensions for better NoteBook development, designed to assist you become a New-Big (a.k.a. 牛逼 or awesome) scholar one day.

In [1]:
#| default_exp nbscholar

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
%load_ext autoreload
%autoreload 2

In [4]:
#| export
from fastcore.script import call_parse



## nbscholar_docs

alternative to nbdev_docs

https://github.com/AnswerDotAI/nbdev/issues/1464





## nbscholar_license

use google addlicense to add license to all files in the repo, while ignoring gitignore and gitmodules.
 

## nbscholar_export

alternative to nbdev_export

In [ ]:
#| export
import configparser
import os
from pathlib import Path

In [9]:
#| export
def read_settings_ini(directory,  item="nbs_path", track="DEFAULT",ini_name='settings.ini'):
    config = configparser.ConfigParser()
    settings_path = os.path.join(directory, ini_name)
    assert os.path.exists(directory), f"Directory {directory} does not exist"
    assert os.path.exists(settings_path), f"Could not find {ini_name} in {directory}"
    config.read(settings_path)
    assert track in config, f"Could not find {track} in {settings_path}"
    assert item in config[track], f"Could not find {item} in {settings_path}"
    return config[track][item]

In [10]:
#| export
import subprocess
import os
from scholarly_infrastructure.logging.nucleus import logger
@call_parse
def nbscholar_export(path:str = "."):
    res = os.system("nbdev_export")
    if res!= 0:
        raise Exception("nbdev_export failed")
    # 读取 settings.ini 的 lib_name
    lib_name = read_settings_ini(path, item="lib_name")

    # MKINIT 生成 __init__.py
    # res = os.system(f"mkinit {lib_name} -w --lazy_loader --recursive --relative")
    res = os.system(f"mkinit {lib_name} -w --lazy_loader_typed --recursive --relative")
    if res!= 0:
        # raise Exception("mkinit failed")
        logger.warning("mkinit failed")
    
    # RUFF 格式化
    res = os.system(f"ruff format {lib_name}")
    if res!= 0:
        logger.warning("ruff format failed")
    
    

## nbscholar_translate

https://github.com/AnswerDotAI/nbdev/issues/1429

## nbscholar_license

https://github.com/AnswerDotAI/nbdev/issues/1475

## nbscholar_separate

https://github.com/AnswerDotAI/nbdev/issues/1468

In [7]:
#| export
import os
import nbformat
import re
from nbformat.notebooknode import NotebookNode, from_dict
from scholarly_infrastructure import default_on_exception

In [13]:
#| export
read_settings_ini_none = default_on_exception(read_settings_ini, default_value=None)


def guess_notebooks_path(directory="."):
    if isinstance(directory, Path):
        directory = directory.as_posix()
    # 读取 setting.ini 的 nbs_path， 如果有的话，没有就返回None
    return read_settings_ini_none(directory)

In [ ]:
from scholarly_infrastructure.help import runs_path

In [ ]:
# 创建一个临时的 settings.ini 文件进行测试
with open(runs_path/"settings.ini", "w") as f:
    f.write("[DEFAULT]\nnbs_path=notebooks")

# 测试 guess_notebooks_path 函数
print(guess_notebooks_path(runs_path))  # 应该输出 'notebooks'

# 删除临时的 settings.ini 文件
os.remove(runs_path/"settings.ini")

# 测试 settings.ini 文件不存在的情况
print(guess_notebooks_path(runs_path))  # 应该输出 None

# 创建一个不包含 nbs_path 的 settings.ini 文件进行测试
with open(runs_path/"settings.ini", "w") as f:
    f.write("[DEFAULT]\nother_key=other_value")

# 测试 guess_notebooks_path 函数
print(guess_notebooks_path(runs_path))  # 应该输出 None

# 删除临时的 settings.ini 文件
os.remove(runs_path/"settings.ini")

notebooks
None
None


In [ ]:
#| export
from copy import deepcopy

In [ ]:
#| export
def split_import_and_code_cells(notebook, inplace=True):
    """
    Process a Jupyter Notebook file, splitting cells with both import and non-import lines into two cells.
    The first new cell will contain only import statements, and the second will contain the rest of the code.
    """
    notebook = notebook if inplace else deepcopy(notebook)
    
    new_cells = []

    for cell in notebook['cells']:
        if cell['cell_type'] == 'code':
            # Split the lines in the cell
            lines = cell['source'].splitlines()

            # Extract leading blank lines or lines starting with "#|"
            leading_lines = []
            while lines and (lines[0].strip() == "" or lines[0].startswith("#|")):
                leading_lines.append(lines.pop(0))

            # Separate import statements and other code lines
            import_lines = [line for line in lines if re.match(r"^\s*import\b|^\s*from\b", line)]
            other_lines = [line for line in lines if line not in import_lines]

            if import_lines and other_lines:
                # Add the leading lines to the import cell
                
                new_cells.append(from_dict(cell | {
                    "cell_type": "code",
                    "metadata": {},
                    "source": "\n".join(leading_lines + import_lines), 
                    "outputs": []
                }))
                # Add the leading lines to the other code cell
                new_cells.append(from_dict(cell | {
                    "cell_type": "code",
                    "metadata": {},
                    "source": "\n".join(leading_lines + other_lines),
                    "outputs": cell['outputs']
                }))
            else:
                # If no split is needed, retain the original cell
                new_cells.append(cell)
        else:
            # Retain non-code cells as is
            new_cells.append(cell)

    # Update the notebook with the modified cells
    notebook['cells'] = new_cells
    return notebook

In [11]:
#| export
def operate_on_notebook_in(input_path, output_path=None, operation=split_import_and_code_cells):
    if output_path is None:
        output_path = input_path
    with open(input_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)
    notebook = operation(notebook)
    # Save the modified notebook
    with open(output_path, 'w', encoding='utf-8') as f:
        nbformat.write(notebook, f)

In [13]:
#| export
def process_notebooks_in_folder(folder_path, operation=split_import_and_code_cells):
    """
    Traverse all .ipynb files in a folder and apply the cell-splitting logic.
    """
    if isinstance(folder_path, Path):
        folder_path = folder_path.as_posix()
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.ipynb'):
                notebook_path = os.path.join(root, file)
                print(f"Processing {notebook_path}")
                operate_on_notebook_in(notebook_path, operation=operation)

In [18]:
#| export
@call_parse
def nbscholar_separate(path:str="."):
    if os.path.isfile(path):
        return operate_on_notebook_in(path, split_import_and_code_cells)
    
    notebook_path = guess_notebooks_path(path)
    if notebook_path is None:
        notebook_path = path
    process_notebooks_in_folder(notebook_path)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()